In [7]:
# import package
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

import matplotlib.pyplot as plt

# set numpy seed for reproducibility
np.random.seed(seed=123)

In [8]:
dataset = [[1.0, 1, -1, -1],
          [-1, -1, 1, 1],
          [-1, -1, 1, 1],
          [1, 1, -1, -1],
          [1, 0, -1, -1],
          [1, 1, 0, -1],
          [0, 1, -1, -1],
          [1, 1, -1, 0]]

In [9]:
x_train = dataset
y_train = dataset

In [10]:
batch_size = len(x_train) # 50
num_epochs = 100
learning_rate = 0.01
size_hidden = 100
n_feature = 4
n_output = 4
batch_no = int(len(x_train) / batch_size)

In [11]:
dim_in = 4
dim_out = 4

In [12]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(dim_in, 100)
        self.fc21 = nn.Linear(100, 20)
        self.fc22 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 100)
        self.fc4 = nn.Linear(100, dim_out)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

net = VAE()

In [15]:
criterion = nn.MSELoss()


def loss_function(output, x, mu, logvar):
    """
    output: output
    x: input
    mu: latent mean
    logvar: latent log variance
    """
    BCE = criterion(output, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [19]:
net(Variable(torch.tensor(x_train[0:1]).float()))

(tensor([[0.0310, 0.1387, 0.0257, 0.1115]], grad_fn=<AddmmBackward>),
 tensor([[ 0.2270, -0.1846, -0.4204, -0.6794,  0.1242, -0.3953,  0.0740, -0.1034,
           0.0511,  0.3149,  0.5216,  0.2645,  0.2418,  0.1490, -0.2263,  0.0878,
          -0.6165, -0.5452,  0.0054, -0.0231]], grad_fn=<AddmmBackward>),
 tensor([[-1.7852e-01, -1.5408e-01, -6.4261e-05,  1.5969e-01, -2.8889e-01,
           3.4127e-02, -1.4518e-01,  4.3779e-01, -1.2346e-02, -2.2461e-01,
           5.3249e-03, -2.9938e-01,  3.2015e-01,  2.0181e-01, -1.3300e-01,
          -1.5935e-01,  5.6809e-01, -5.3227e-02,  1.4094e-01, -3.5000e-01]],
        grad_fn=<AddmmBackward>))

In [21]:
running_loss = 0.0

for epoch in range(num_epochs):
    
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.tensor(x_train[start:end]).float())
        labels = Variable(torch.tensor(y_train[start:end]).float())
        
        #net.train()
        #train_loss = 0
    
        optimizer.zero_grad()
        # recon_batch
        output_batch, mu, logvar = net(inputs)
        loss = loss_function(output_batch, inputs, mu, logvar)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    
    if epoch % 10 == 0:    
        print('Epoch {}'.format(epoch+1), "loss: ", running_loss)
    running_loss = 0.0

Epoch 1 loss:  9.872776985168457
Epoch 11 loss:  1.537191390991211
Epoch 21 loss:  1.0197230577468872
Epoch 31 loss:  0.7133520841598511
Epoch 41 loss:  0.7885348200798035
Epoch 51 loss:  0.7978350520133972
Epoch 61 loss:  0.6025741696357727
Epoch 71 loss:  1.0374019145965576
Epoch 81 loss:  0.7229259014129639
Epoch 91 loss:  0.6768777966499329


In [24]:
pred, _, __ = net(Variable(torch.tensor([[1, 0, 0, 0]]).float()))
print (pred)

tensor([[ 0.5354,  0.5857, -0.2769, -0.5899]], grad_fn=<AddmmBackward>)


In [25]:
pred, _, __ = net(Variable(torch.tensor([[1, 0, 0, 0]]).float()))
print (pred.data)

tensor([[ 0.7115,  0.4597, -0.4149, -0.1969]])


In [26]:
pred, _, __ = net(Variable(torch.tensor([[0, 1, 0, 0]]).float()))
print (pred.data)

tensor([[ 0.2998,  0.3850, -0.4192, -0.4222]])


In [27]:
pred, _, __ = net(Variable(torch.tensor([[0, 0, -1, 0]]).float()))
print (pred.data)

tensor([[ 0.2683,  0.1296, -0.2164, -0.0694]])


In [28]:
pred, _, __ = net(Variable(torch.tensor([[0, 0, 0, -1]]).float()))
print (pred.data)

tensor([[ 0.0533,  0.0750, -0.0284, -0.0880]])


In [30]:
__

tensor([[ 0.0436,  0.0459, -0.1022, -0.0577,  0.0004,  0.0234,  0.0432, -0.1221,
          0.0405,  0.0554, -0.0554,  0.0557,  0.0374,  0.0171,  0.1135,  0.1049,
         -0.0231, -0.0089,  0.0698,  0.0623]], grad_fn=<AddmmBackward>)